In [1]:
import urllib.request as url
import ssl
from multiprocessing import Pool
from bs4 import BeautifulSoup as BS
import time
import os
import re

# Obter atas do Copom
### Repositório do Github: https://github.com/paulodt/TCC-PauloDeTarso

## Códigos auxiliares

Código para transformar os pdfs em texto. Criado a partir da modificação de:

http://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO

def convert_pdf(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    fp = open(path, 'rb')
    process_pdf(rsrcmgr, device, fp)
    fp.close()
    device.close()

    str = retstr.getvalue()
    retstr.close()
    return str

Código para remover símbolos das strings

In [107]:
def clean_string(s):
    #Remover símbolos
    s = re.sub(r'[^\w]', ' ', s)
    #Trocar espaço múltiplo por simples
    s = re.sub( '\s+', ' ', s)
    return s

## Baixar as atas do COPOM

Para ignorar a verificação de certificados, me baseio em:
http://stackoverflow.com/questions/36600583/python-3-urllib-ignore-ssl-certificate-verification

In [108]:
#Ignorar a verificação de certificados no "urlopen"
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

Se a ata estiver em pdf, utilizo o programa "convert_pdf" acima para transformar em um txt. Se for um html, transformo em um txt diretamente.

In [ ]:
#Parâmetros
inicio_atas = 21
final_atas = 206
total_atas = (final_atas - inicio_atas + 1) + 1
atas = range(inicio_atas,final_atas+1)

path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

for n_ata in atas:
    #Progresso
    print("Progresso: " + str(round(100*(n_ata-inicio_atas+1)/total_atas)) + "%",end="\r")

    #Parâmetros
    ata = str(n_ata)
    link = "https://www.bcb.gov.br/?COPOM" + ata
    filename = "COPOM_" + ata

    #Acessar o site e obter o tipo (html ou PDF)
    site = url.urlopen(link,context=ctx)
    content_type = dict(site.getheaders())["Content-Type"]
    if "pdf" not in content_type:
        full_text = site.read()
        full_text = "\n".join(BS(full_text,"lxml").findAll(text=True))
        clean_text = full_text[full_text.index("Data"):]

        file_out = open(path + filename + ".txt","wb")
        file_out.write(clean_text.encode("utf-8",errors="replace"))
        file_out.close()
        
    else:
        #Baixar o arquivo
        with open(path + filename + ".pdf","wb") as file_in:
            file_in.write(site.read())
        file_in.close()

        full_text = convert_pdf(path + filename + ".pdf")
        full_text = full_text[full_text.index("Data"):]

        #Escrever no arquivo txt
        file_out = open(path + filename + ".txt","wb")
        file_out.write(full_text.encode("utf-8"))
        file_out.close()

        #Remover o arquivo original
        os.remove(filename + ".pdf")

In [ ]:
for file in os.listdir(path):
    if open(path + file).readline() == "":
        print("O arquivo " + file + " está vazio!")